# Fish Net Classification

In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # specify which GPU(s) to be used

import torch.backends.cudnn as cudnn
cudnn.benchmark = True
from utils import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

data_path = "/media/slowSSD/Carribean_Challenge/data/fish"

## (Optional: Copy to Devbox)

### Copy net images and no net images

In [24]:
# shutil.rmtree("{}/nets".format(data_path))

In [25]:
# os.makedirs("{}/nets".format(data_path), exist_ok=True)
# os.makedirs("{}/no_nets".format(data_path), exist_ok=True)

In [26]:
# for path in tqdm(glob.glob("data/nets/*")):
#     shutil.copyfile(path, path.replace("data/nets", "{}/nets".format(.format(data_path))))

100%|██████████| 112/112 [00:30<00:00,  3.69it/s]


In [19]:
# for path in tqdm(glob.glob("data/no_nets/*")):
#     shutil.copyfile(path, path.replace("data/no_nets", "{}/no_nets".format(.format(data_path))))

100%|██████████| 490/490 [00:20<00:00, 23.35it/s]


### Copy inference images that are not in the area of training images

In [3]:
# coords = [j.split("/")[-1].split("_")[1] for j in glob.glob("data/no_nets/*")] + [
#     j.split("/")[-1].split("_")[1] for j in glob.glob("data/nets/*")]
# training_set_coordinates = []
# for coord in coords:
#     lat, long = map(float, coord.split(","))
#     training_set_coordinates.append((lat, long))

In [1]:
# for path in tqdm(glob.glob("data/inference_random/*")):
#     use = True
#     coord = path.split("/")[-1].split("_")[1]
#     new_lat, new_long = map(float, coord.split(","))
#     for lat, long in training_set_coordinates:
#         if abs(new_lat-lat)<LAT_PER_KM*3 and abs(new_long-long)<LONG_PER_KM*3:
#             print("Discard Lat {:.3f} and Long {:.3f} as its too close to {:.3f} and {:.3f}".format(new_lat, new_long, lat, long))
#             use = False
#             break
#     if use:
#         print("Use Lat {:.3f} and Long {:.3f}".format(new_lat, new_long))
#         shutil.copyfile(path, path.replace("data/inference_random", "{}/no_nets".format(data_path)))

## Resizing

In [4]:
images = glob.glob("{}/nets/*.png".format(data_path))
labels = [1]*len(images)
how_many_nets = len(labels)
print("{:3} images with nets present".format(how_many_nets))

images += glob.glob("{}/no_nets/*.png".format(data_path))
how_many_without_nets = len(images)-how_many_nets
labels += [0]*how_many_without_nets
print("{:3} images without nets present".format(how_many_without_nets))

  0 images with nets present
  0 images without nets present


In [17]:
img = Image.open(images[-1])
img.size

In [13]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing
cpus = multiprocessing.cpu_count()
cpus = min(48, cpus)
print("{} cpus".format(cpus))

32 cpus


In [14]:
# sz = (450, 432)
sz = (900, 864)

In [15]:
replace_string = 'nets'
replace_string_with = "nets_{}x{}".format(sz[0], sz[1])

In [16]:
shutil.rmtree("{}/nets_{}x{}".format(data_path, sz[0], sz[1]))
shutil.rmtree("{}/no_nets_{}x{}".format(data_path, sz[0], sz[1]))

os.makedirs("{}/nets_{}x{}".format(data_path, sz[0], sz[1]), exist_ok=True)
os.makedirs("{}/no_nets_{}x{}".format(data_path, sz[0], sz[1]), exist_ok=True)

In [17]:
def resize_img(im, fn):
    im = im.resize(sz, resample=Image.BICUBIC)
    im.save(fn.replace(replace_string, replace_string_with))

def resizer_img(fn):
    im = Image.open(fn).convert("RGB")
    print("{:50} -> {:50}: {}".format(fn, fn.replace(replace_string, replace_string_with), im.size))
    resize_img(im, fn)

def resize_imgs(img_paths):
    with ProcessPoolExecutor(cpus) as e: e.map(resizer_img, img_paths)

In [18]:
resize_imgs(images)

/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_50.309,-125.317_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_50.309,-125.317_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_50.415,-125.66_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_50.415,-125.66_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_49.797,-124.098_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_49.797,-124.098_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_49.656,-126.431_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_49.656,-126.431_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_49.513,-123.492_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_49.513,-123.492_zoom16.png: (1800, 172

/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_49.327,-125.799_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_49.327,-125.799_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_49.629,-123.845_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_49.629,-123.845_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_50.854,-126.759_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_50.854,-126.759_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_50.324,-125.26_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_50.324,-125.26_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_50.608,-126.363_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_50.608,-126.363_zoom16.png: (1800, 172

/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_50.65,-126.618_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_50.65,-126.618_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_50.797,-126.496_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_50.797,-126.496_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_49.014,-125.03_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_49.014,-125.03_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_49.404,-126.152_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_49.404,-126.152_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/nets/2019-11-17_50.533,-126.226_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/nets_900x864/2019-11-17_50.533,-126.226_zoom16.png: (1800, 1726)

/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_52.117,-123.897_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_52.117,-123.897_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.066,-127.574_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.066,-127.574_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_52.26,-126.208_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_52.26,-126.208_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_50.223,-127.257_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_50.223,-127.257_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_48.764,-125.368_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_48.764,-

/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_52.684,-126.079_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_52.684,-126.079_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_52.446,-127.974_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_52.446,-127.974_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_52.337,-125.122_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_52.337,-125.122_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_51.427,-125.934_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_51.427,-125.934_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.581,-127.109_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.581

...

/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.963,-127.805_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.963,-127.805_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_48.621,-126.294_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_48.621,-126.294_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.501,-126.97_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.501,-126.97_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_50.329,-124.528_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_50.329,-124.528_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.403,-127.999_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.403,-

/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_48.841,-124.668_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_48.841,-124.668_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_51.768,-127.619_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_51.768,-127.619_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_52.782,-125.599_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_52.782,-125.599_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_48.725,-123.901_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_48.725,-123.901_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_51.141,-128.313_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_51.141

/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_50.255,-123.939_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_50.255,-123.939_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.277,-126.817_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.277,-126.817_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_50.099,-126.475_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_50.099,-126.475_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_51.659,-124.139_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_51.659,-124.139_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_50.513,-127.915_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_50.513

/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_52.687,-124.209_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_52.687,-124.209_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_50.975,-127.327_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_50.975,-127.327_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.65,-125.454_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.65,-125.454_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.903,-126.661_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.903,-126.661_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.491,-126.087_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.491,-

/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.473,-124.207_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.473,-124.207_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_51.749,-127.257_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_51.749,-127.257_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_48.82,-124.947_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_48.82,-124.947_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.109,-124.261_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.109,-124.261_zoom16.png: (1800, 1726)
/media/slowSSD/Carribean_Challenge/data/fish/no_nets/2019-11-17_49.818,-127.912_zoom16.png -> /media/slowSSD/Carribean_Challenge/data/fish/no_nets_900x864/2019-11-17_49.818,-

## Training - much more data, 900x864

In [19]:
# sz = (450, 432)
sz = (900, 864)
random_crop = 448
val_percentage = 0.15

# IMAGES WITH NETS
images = glob.glob("{}/nets_{}x{}/*.png".format(data_path, sz[0], sz[1]))
labels = [1]*len(images)
how_many_nets = len(labels)
print("{:3} images with nets present".format(how_many_nets))

# VAL/TRAIN SPLIT
np.random.seed(1338)
np.random.shuffle(images)
np.random.seed(1338)
np.random.shuffle(labels)

train_images = images[:int((1-val_percentage)*len(images))]
train_labels = labels[:int((1-val_percentage)*len(labels))]
val_images = images[int((1-val_percentage)*len(images)):]
val_labels = labels[int((1-val_percentage)*len(labels)):]

# IMAGES WITHOUT NETS
images = glob.glob("{}/no_nets_{}x{}/*.png".format(data_path, sz[0], sz[1]))
how_many_without_nets = len(images)
labels = [0]*how_many_without_nets
print("{:3} images without nets present".format(how_many_without_nets))

np.random.seed(1339)
np.random.shuffle(images)
np.random.seed(1339)
np.random.shuffle(labels)

train_images += images[:int((1-val_percentage)*len(images))]
train_labels += labels[:int((1-val_percentage)*len(labels))]
val_images += images[int((1-val_percentage)*len(images)):]
val_labels += labels[int((1-val_percentage)*len(labels)):]

print("#"*60)
print("{:3} Train images --> {:2} nets = {:.1f}%".format(len(train_images), sum(train_labels), 100*sum(train_labels)/len(train_images)))
print("{:3} Valid images --> {:2} nets = {:.1f}%".format(len(val_images), sum(val_labels), 100*sum(val_labels)/len(val_images)))

111 images with nets present
7531 images without nets present
############################################################
6495 Train images --> 94 nets = 1.4%
1147 Valid images --> 17 nets = 1.5%


In [20]:
train_dataset_raw = SalmonDataset(images=train_images, targets=train_labels) 

train_transforms = []
# train_transforms.append(albumentations.RandomRotate90(p=0.5))
# train_transforms.append(albumentations.HorizontalFlip(p=0.5))
# train_transforms.append(albumentations.VerticalFlip(p=0.5))
# train_transforms.append(albumentations.Transpose(p=0.5))
# train_transforms.append(albumentations.SmallestMaxSize(max_size=256, interpolation=cv2.INTER_CUBIC))
train_transforms.append(albumentations.RandomCrop(random_crop, random_crop))
train_dataset = SalmonDataset(images=train_images, targets=train_labels, transforms=albumentations.Compose(train_transforms)) 

val_transforms = []
# val_transforms.append(albumentations.SmallestMaxSize(max_size=256, interpolation=cv2.INTER_CUBIC))
# val_transforms.append(albumentations.CenterCrop(224, 224))
val_dataset = SalmonDataset(images=val_images, targets=val_labels, transforms=albumentations.Compose(val_transforms)) 

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=10, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=10, pin_memory=True)

__Visualize__

In [21]:
for j in range(10):
    rand_int = np.random.randint(len(train_dataset))
    f, axarr = plt.subplots(1, 4, figsize=(30, 12))
    sample = train_dataset_raw[rand_int]
    axarr[0].imshow(np.transpose((sample["image"].numpy()*255).astype(np.uint8), (1,2,0)))
    if sample["target"]==1: axarr[0].set_title(classes[0])
    else: axarr[0].set_title("No net")
    print(train_images[rand_int])
    axarr[1].imshow(np.transpose((train_dataset[rand_int]["image"].numpy()*255).astype(np.uint8), (1,2,0)))
    axarr[2].imshow(np.transpose((train_dataset[rand_int]["image"].numpy()*255).astype(np.uint8), (1,2,0)))
    axarr[3].imshow(np.transpose((train_dataset[rand_int]["image"].numpy()*255).astype(np.uint8), (1,2,0)))
    plt.show()

In [22]:
epochs = 1000
print_every_x_batches = 1000
lr = 0.001
arch = "resnet18"

In [23]:
patience = 3
model_name = "{}_{}nets_{}wo_val_v1_resize_{}x{}_rc_{}_pat{}".format(arch, sum(train_labels), len(train_labels)-sum(train_labels), sz[0], sz[1], random_crop, patience)
print(model_name)
main(arch=arch, model_name=model_name, train_loader=train_loader, val_loader=val_loader, epochs=epochs, lr=lr, patience=patience, print_every_x_batches=print_every_x_batches)
torch.cuda.empty_cache()

resnet18_94nets_6401wo_val_v1_resize_900x864_rc_448_pat3
Epoch: [0]	Time  0.173	Data  0.009	Loss 8.2861e-02	Acc@1 0.9800
Val: Loss 0.0641 Acc@1 0.9852
Epoch: [1]	Time  0.157	Data  0.009	Loss 5.7166e-02	Acc@1 0.9851
Val: Loss 0.0727 Acc@1 0.9852
Epoch: [2]	Time  0.159	Data  0.010	Loss 5.7423e-02	Acc@1 0.9848
Val: Loss 0.0581 Acc@1 0.9852
Epoch: [3]	Time  0.158	Data  0.009	Loss 5.0033e-02	Acc@1 0.9848
Val: Loss 0.0580 Acc@1 0.9843
Epoch: [4]	Time  0.158	Data  0.009	Loss 4.8404e-02	Acc@1 0.9846
Val: Loss 0.0706 Acc@1 0.9808
Epoch: [5]	Time  0.159	Data  0.010	Loss 5.2654e-02	Acc@1 0.9846
Val: Loss 0.1485 Acc@1 0.9582
Epoch: [6]	Time  0.159	Data  0.009	Loss 4.2602e-02	Acc@1 0.9860
Val: Loss 0.0606 Acc@1 0.9852
Epoch: [7]	Time  0.159	Data  0.009	Loss 4.3537e-02	Acc@1 0.9865
Val: Loss 0.0475 Acc@1 0.9869
Epoch: [8]	Time  0.159	Data  0.010	Loss 3.0870e-02	Acc@1 0.9909
Val: Loss 0.0547 Acc@1 0.9843
Epoch: [9]	Time  0.159	Data  0.010	Loss 2.9969e-02	Acc@1 0.9912
Val: Loss 0.0620 Acc@1 0.9852
Epo

In [6]:
patience = 3
model_name = "{}_{}nets_{}wo_val_v1_resize_{}x{}_rc_{}_pat{}".format(arch, sum(train_labels), len(train_labels)-sum(train_labels), sz[0], sz[1], random_crop, patience)
print(model_name)
main(arch=arch, model_name=model_name, train_loader=train_loader, val_loader=val_loader, epochs=epochs, lr=lr, patience=patience, print_every_x_batches=print_every_x_batches)
torch.cuda.empty_cache()

resnet18_88nets_6405wo_val_v1_resize_900x864_rc_448_pat3
Epoch: [0]	Time  0.173	Data  0.009	Loss 8.0225e-02	Acc@1 0.9730
Val: Loss 0.0585 Acc@1 0.9817
Epoch: [1]	Time  0.158	Data  0.008	Loss 5.0500e-02	Acc@1 0.9855
Val: Loss 0.0636 Acc@1 0.9861
Epoch: [2]	Time  0.159	Data  0.009	Loss 4.9605e-02	Acc@1 0.9843
Val: Loss 0.0532 Acc@1 0.9861
Epoch: [3]	Time  0.159	Data  0.009	Loss 4.6608e-02	Acc@1 0.9848
Val: Loss 0.0477 Acc@1 0.9861
Epoch: [4]	Time  0.159	Data  0.009	Loss 3.5710e-02	Acc@1 0.9897
Val: Loss 0.0644 Acc@1 0.9861
Epoch: [5]	Time  0.159	Data  0.009	Loss 3.5356e-02	Acc@1 0.9897
Val: Loss 0.0542 Acc@1 0.9861
Epoch: [6]	Time  0.159	Data  0.009	Loss 3.0493e-02	Acc@1 0.9895
Val: Loss 0.0541 Acc@1 0.9861
Epoch: [7]	Time  0.160	Data  0.009	Loss 2.5221e-02	Acc@1 0.9934
Val: Loss 0.0602 Acc@1 0.9861
Epoch     7: reducing learning rate of group 0 to 5.0000e-04.
Epoch: [8]	Time  0.160	Data  0.009	Loss 1.9763e-02	Acc@1 0.9938
Val: Loss 0.0488 Acc@1 0.9869
Epoch: [9]	Time  0.160	Data  0.009	

In [25]:
patience = 3
model_name = "{}_74nets_490wo_val_v1_resize_{}x{}_rc_{}_pat{}".format(arch, sz[0], sz[1], random_crop, patience)
print(model_name)
main(arch=arch, model_name=model_name, train_loader=train_loader, val_loader=val_loader, epochs=epochs, lr=lr, patience=patience, print_every_x_batches=print_every_x_batches)
torch.cuda.empty_cache()

resnet18_74nets_490wo_val_v1_resize_900x864_rc_448_pat3
Epoch: [0]	Time  0.684	Data  0.347	Loss 3.7674e-01	Acc@1 0.8581
Val: Loss 0.6430 Acc@1 0.8407
Epoch: [1]	Time  0.274	Data  0.131	Loss 2.1250e-01	Acc@1 0.9268
Val: Loss 6.5540 Acc@1 0.2832
Epoch: [2]	Time  0.271	Data  0.131	Loss 1.8606e-01	Acc@1 0.9357
Val: Loss 0.3224 Acc@1 0.8673
Epoch: [3]	Time  0.468	Data  0.333	Loss 2.0783e-01	Acc@1 0.9335
Val: Loss 0.3761 Acc@1 0.8673
Epoch: [4]	Time  0.302	Data  0.159	Loss 1.6783e-01	Acc@1 0.9424
Val: Loss 0.3430 Acc@1 0.8407
Epoch: [5]	Time  0.276	Data  0.134	Loss 1.7272e-01	Acc@1 0.9379
Val: Loss 0.5113 Acc@1 0.7699
Epoch: [6]	Time  0.286	Data  0.145	Loss 1.4622e-01	Acc@1 0.9512
Val: Loss 0.4424 Acc@1 0.8673
Epoch     6: reducing learning rate of group 0 to 5.0000e-04.
Epoch: [7]	Time  0.273	Data  0.130	Loss 1.3133e-01	Acc@1 0.9690
Val: Loss 0.2358 Acc@1 0.9027
Epoch: [8]	Time  0.286	Data  0.143	Loss 2.0953e-01	Acc@1 0.9468
Val: Loss 0.2208 Acc@1 0.9204
Epoch: [9]	Time  0.284	Data  0.144	L